<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/et_tiny_finetune_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U accelerate
! pip install -U transformers
!pip install datasets
!pip install evaluate
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflict

In [3]:
pip install openai-whisper

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import argparse
import evaluate
from dataclasses import dataclass
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from typing import Any, Dict, List, Union
from datasets import DatasetDict, Audio, load_from_disk, concatenate_datasets
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:


#lt_voxpopuli_dataset = load_dataset("facebook/voxpopuli", "lt", split=['train', 'test'])

NameError: name 'load_dataset' is not defined

In [5]:
from datasets import load_dataset, load_metric

et_voxpopuli_dataset = load_dataset("facebook/voxpopuli", "et", split=['train', 'test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
train_dataset = et_voxpopuli_dataset[0].remove_columns(['audio_id', 'language', 'raw_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'])


In [7]:
test_dataset = et_voxpopuli_dataset[1].remove_columns(['audio_id', 'language', 'raw_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'])


In [ ]:
#rom datasets import load_dataset, load_metric
#voxpopuli_dataset = load_dataset("facebook/voxpopuli", "lt", split=['train', 'test'])
#voxpopuli_dataset = load_dataset("facebook/voxpopuli", "et")

In [ ]:
#dataset_lt = voxpopuli_dataset.remove_columns(['audio_id', 'language', 'raw_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'])
#dataset_lt = voxpopuli_dataset[0].remove_columns(['audio_id', 'language', 'raw_text', 'speaker_id', 'is_gold_transcript', 'accent'])

In [8]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
gradient_checkpointing = True
freeze_feature_encoder = False
freeze_encoder = False

do_normalize_eval = True
do_lower_case = False
do_remove_punctuation = False
normalizer = BasicTextNormalizer()


In [24]:
from dataclasses import dataclass

from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments,Seq2SeqTrainer

model_checkpoint= "openai/whisper-tiny"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_checkpoint)
tokenizer = WhisperTokenizer.from_pretrained(model_checkpoint, language="Estonian", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_checkpoint, language="Estonian", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)

if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

if freeze_feature_encoder:
    model.freeze_feature_encoder()

if freeze_encoder:
    model.freeze_encoder()
    model.model.encoder.gradient_checkpointing = False


model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

if gradient_checkpointing:
    model.config.use_cache = False

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [25]:
model_checkpoint_name = model_checkpoint.split("/")[-1]
repo_name = f"{model_checkpoint_name}-demo-colab"

In [26]:
def prepare_dataset(batch):
    # Load and resample audio data to the expected sampling rate
    audio = batch["audio"]
    input_features = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    #input_features = input_features.reshape(-1, 80, 3000)

    # Ensure the last dimension of input_features is 3000
    if input_features.shape[-1] < 3000:
        padding = torch.zeros(3000 - input_features.shape[-1])
        input_features = torch.cat([input_features, padding], dim=0)

    batch["input_features"] = input_features

    # Compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # Optional pre-processing steps
    transcription = batch["normalized_text"]
    if do_lower_case:
        transcription = transcription.lower()
    if do_remove_punctuation:
        transcription = normalizer(transcription).strip()

    # Encode target text to label ids
    batch["labels"] = processor.tokenizer(transcription, padding="max_length", max_length=max_label_length).input_ids

    return batch

max_label_length = model.config.max_length
min_input_length = 0.0
max_input_length = 30.0
def is_in_length_range(length, labels):
    return min_input_length < length < max_input_length and 0 < len(labels) < max_label_length

In [27]:
train_dataset = train_dataset.map(prepare_dataset, batch_size=2)
#test_dataset = test_dataset.map(prepare_dataset, batch_size=8)


Map:   0%|          | 0/834 [00:00<?, ? examples/s]

In [29]:
#train_dataset = train_dataset.map(prepare_dataset, batch_size=8)
test_dataset = test_dataset.map(prepare_dataset, batch_size=2)


Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:

# Apply preprocessing and ensure 'labels' key is added
#dataset_lt1 = dataset_lt.map(prepare_dataset, batch_size=8)

In [ ]:
#y=dataset_it1["train"]["input_features"][0]

In [ ]:
#import numpy as np
#y=np.array(y)
#y.shape

NameError: name 'y' is not defined

In [30]:
from typing import Any, Dict, List, Union
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print('DATASET PREPARATION COMPLETED')

DATASET PREPARATION COMPLETED


In [32]:
import numpy as np
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [33]:
model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (fin

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Attention mechanism
class WhisperAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.k_proj = nn.Linear(d_model, d_model, bias=False)
        self.v_proj = nn.Linear(d_model, d_model, bias=True)
        self.q_proj = nn.Linear(d_model, d_model, bias=True)
        self.out_proj = nn.Linear(d_model, d_model, bias=True)

    def forward(self, k, v, q):
        k = self.k_proj(k)
        v = self.v_proj(v)
        q = self.q_proj(q)

        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(k.size(-1), dtype=torch.float32))
        attn_probs = F.softmax(attn_scores, dim=-1)

        context = torch.matmul(attn_probs, v)
        return self.out_proj(context)

# Encoder layer
class WhisperEncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.self_attn = WhisperAttention(d_model)
        self.self_attn_layer_norm = nn.LayerNorm(d_model)
        self.activation_fn = nn.GELU()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.final_layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        attn_output = self.self_attn(x, x, x)
        x = self.self_attn_layer_norm(x + attn_output)

        fc_output = self.fc2(self.activation_fn(self.fc1(x)))
        x = self.final_layer_norm(x + fc_output)
        return x

# Encoder
class WhisperEncoder(nn.Module):
    def __init__(self, input_dim, d_model, n_layers, d_ff, max_len):
        super().__init__()
        self.conv1 = nn.Conv1d(input_dim, d_model, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(d_model, d_model, kernel_size=3, stride=2, padding=1)
        self.embed_positions = nn.Embedding(max_len, d_model)
        self.layers = nn.ModuleList([WhisperEncoderLayer(d_model, d_ff) for _ in range(n_layers)])
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        # Transpose to match the (batch_size, seq_len, features) format
        x = x.transpose(1, 2)  # now shape: (batch_size, seq_len, d_model)

        # Adjusting the position_ids based on the actual sequence length after convolution
        position_ids = torch.arange(x.size(1), dtype=torch.long, device=x.device)
        position_embeddings = self.embed_positions(position_ids)  # now shape: (seq_len, d_model)

        x += position_embeddings

        for layer in self.layers:
            x = layer(x)

        x = self.layer_norm(x)
        return x

# Decoder layer
class WhisperDecoderLayer(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.self_attn = WhisperAttention(d_model)
        self.self_attn_layer_norm = nn.LayerNorm(d_model)
        self.encoder_attn = WhisperAttention(d_model)
        self.encoder_attn_layer_norm = nn.LayerNorm(d_model)
        self.activation_fn = nn.GELU()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.final_layer_norm = nn.LayerNorm(d_model)

    def forward(self, x, encoder_output):
        self_attn_output = self.self_attn(x, x, x)
        x = self.self_attn_layer_norm(x + self_attn_output)

        enc_attn_output = self.encoder_attn(encoder_output, encoder_output, x)
        x = self.encoder_attn_layer_norm(x + enc_attn_output)

        fc_output = self.fc2(self.activation_fn(self.fc1(x)))
        x = self.final_layer_norm(x + fc_output)
        return x
# Decoder
class WhisperDecoder(nn.Module):
    def __init__(self, d_model, n_layers, d_ff, max_len, vocab_size):
        super().__init__()
        self.embed_tokens = nn.Embedding(vocab_size, d_model, padding_idx=50257)
        self.embed_positions = nn.Embedding(max_len, d_model)
        self.layers = nn.ModuleList([WhisperDecoderLayer(d_model, d_ff) for _ in range(n_layers)])
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x, encoder_output):
        position_ids = torch.arange(x.size(1), dtype=torch.long, device=x.device)
        x = self.embed_tokens(x) + self.embed_positions(position_ids)

        for layer in self.layers:
            x = layer(x, encoder_output)

        x = self.layer_norm(x)
        return x

# Complete model
class WhisperForfinetune(nn.Module):
    def __init__(self, input_dim=80, encoder_d_model=512, encoder_n_layers=6, encoder_d_ff=2048, max_len=1500, vocab_size=51865):
        super().__init__()
        self.encoder = WhisperEncoder(input_dim, encoder_d_model, encoder_n_layers, encoder_d_ff, max_len)
        self.decoder = WhisperDecoder(encoder_d_model, encoder_n_layers, encoder_d_ff, max_len, vocab_size)
        self.proj_out = nn.Linear(encoder_d_model, vocab_size, bias=False)

    def forward(self, input_features, labels):
        encoder_output = self.encoder(input_features)
        decoder_output = self.decoder(labels, encoder_output)
        logits = self.proj_out(decoder_output)

        outputs = {'logits': logits}
        if labels is not None:
          loss_fn = nn.CrossEntropyLoss()
          # Reshape labels and logits to compute loss, adjust dimensions as necessary
          loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
          outputs['loss'] = loss

        return outputs

# Instantiate the model
model1 = WhisperForfinetune()

# Example input (dummy data)
#input_features = torch.randn(1, 80, 3000)  # Batch size x Input dimension x Sequence length
#labels = torch.randint(0, 51865, (1, 100))  # Example target token sequence

# Forward pass
#output = model1(input_features, labels)


In [35]:
import torch
import gc

# Before starting the training, clear any residual memory
gc.collect()
torch.cuda.empty_cache()

In [36]:
from transformers import TrainingArguments

training_args = Seq2SeqTrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=2,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  gradient_checkpointing=False,
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=False,
)

In [37]:
from transformers import Trainer
trainer = Seq2SeqTrainer(
    model=model1,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [38]:
import torch
torch.cuda.empty_cache()

trainer.train()

Step,Training Loss,Validation Loss,Wer
50,8.751000,4.026247,1.000000
100,2.170800,1.765254,1.000000
150,1.736100,1.636666,1.000000
200,1.573400,1.298401,1.000000
250,1.153000,0.958139,0.971338
300,0.819600,0.717712,0.945860
350,0.684100,0.538836,0.859873
400,0.465900,0.405791,0.772824
450,0.352300,0.321041,0.692144
500,0.273500,0.263976,0.583864


TrainOutput(global_step=4160, training_loss=0.23802275513522686, metrics={'train_runtime': 6713.2843, 'train_samples_per_second': 2.485, 'train_steps_per_second': 0.62, 'total_flos': 0.0, 'train_loss': 0.23802275513522686, 'epoch': 19.95})

In [23]:
trainer.evaluate()

{'eval_loss': 0.018378425389528275,
 'eval_wer': 0.0435244161358811,
 'eval_runtime': 45.2705,
 'eval_samples_per_second': 1.127,
 'eval_steps_per_second': 0.155,
 'epoch': 19.95}